# Hardpoint Breakaway Tests - Recent History

This notebook evaluates the hardpoint breakaway tests performed on a given `day_obs`.  

In [ ]:
# Times Square Parameters
day_obs = 20250804  # YYYYMMDD

In [ ]:
import asyncio
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import matplotlib.dates as mdates

from astropy.time import Time
from ipywidgets import widgets
from IPython.display import HTML
from typing import Optional

from lsst_efd_client import EfdClient
from lsst.ts.xml.enums.MTM1M3 import HardpointTest

from lsst.summit.utils.efdUtils import (
    getEfdData,
    getDayObsEndTime,
    getDayObsStartTime,
    makeEfdClient,
)

In [ ]:
# Number of hardpoints
n_hardpoints = 6

# HP BreakAway Limits from https://sitcomtn-082.lsst.io/
COMPRESSION_LOWER_LIMIT = 2981  # N
COMPRESSION_UPPER_LIMIT = 3959  # N
TENSION_LOWER_LIMIT = -4420  # N
TENSION_UPPER_LIMIT = -3456  # N

# Used to select the window size near zero measured forces.
DISPLACEMENT_CROP_RANGE = 300  # um
DISPLACEMENT_CROP_RANGE_FOR_FIT = 100  # um

# Used to determine if a test was valid or not
MEASURED_FORCE_MAXIMUM_TOLERANCE = 1000  # N

# Meter to micrometer
METER_TO_MICROMETER = 1e6 # um/m

# Maximum spec stiffness
SPEC_STIFFNESS = 100  # N/um

# Semaphore to control the number of tasks we can run in parallel
#  Tune between 4 and 8 (thanks ChatGPT)
SEMAPHORE_COUNTER = 4

# Create an EFD client instance
client = makeEfdClient()

# Create semaphore for parallel task control
sem = asyncio.Semaphore(SEMAPHORE_COUNTER)

# When running in Times Square, start and end day obs are the same.
#   However, you can pick more days here if you want.
start_day_obs = day_obs
end_day_obs = day_obs

# Get the start and end times
start_time = getDayObsStartTime(start_day_obs)
end_time = getDayObsEndTime(end_day_obs)

In [ ]:
# Query the CSC level command that we can use to define the start of a
# breakaway test for a given strut.
test_command_df = getEfdData(
    client=client,
    topic="lsst.sal.MTM1M3.command_testHardpoint",
    columns=["hardpointActuator"],
    begin=start_time,
    end=end_time,
)

In [ ]:
# Query the status of the breakaway tests as they evolve.
test_status_df = getEfdData(
    client=client,
    topic="lsst.sal.MTM1M3.logevent_hardpointTestStatus",
    columns=[f"testState{i}" for i in range(n_hardpoints)],
    begin=start_time,
    end=end_time,
)

In [ ]:
# Container for results
test_windows = []

# End reasons
end_reasons = [HardpointTest.NOTTESTED, HardpointTest.PASSED, HardpointTest.FAILED]

# Iterate over each test start command
for t_begin, row in test_command_df.iterrows():
    hp = row["hardpointActuator"]  # 1-indexed
    col_name = f"testState{hp - 1}"

    if col_name in test_status_df.columns:
        # Slice status df from the start_time
        slice_begin = test_status_df[col_name].loc[t_begin:]

        positive_mask = slice_begin.isin([HardpointTest.TESTINGPOSITIVE])
        positive_timestamp = slice_begin[positive_mask].first_valid_index()
        positive_slice = test_status_df[col_name].loc[positive_timestamp:]

        negative_mask = positive_slice.isin([HardpointTest.TESTINGNEGATIVE])
        negative_timestamp = positive_slice[negative_mask].first_valid_index()
        negative_slice = test_status_df[col_name].loc[negative_timestamp:]

        end_mask = negative_slice.isin(end_reasons)
        end_timestamp = negative_slice[end_mask].first_valid_index()
        end_reason = test_status_df[col_name].loc[end_timestamp]

        if end_reason not in [HardpointTest.PASSED, HardpointTest.FAILED]:
            print(
                f"Warning - Could not find end of tests for HP{hp} that started on {Time(t_begin).iso}"
            )

        # Create dictionary
        test_windows.append(
            {
                "hardpoint": hp,
                "t_begin": t_begin,
                "t_positive": positive_timestamp,
                "t_negative": negative_timestamp,
                "t_end": end_timestamp,
                "end_reason": end_reason,
            }
        )

# Convert into a DataFrame for convenience
test_windows_df = pd.DataFrame(test_windows)

if test_command_df.empty:
    raise ValueError(
        f"Failed to find any commanded hardpoint breakaway tests for day_obs {day_obs}.\n"
        f"  Queries started from {start_time} to {end_time}."
    )

In [ ]:
def query_tel_single_hp(
    client: EfdClient, single_hp_test: pd.Series, sampling: str = "500ms"
) -> pd.DataFrame:
    hp_index = single_hp_test["hardpoint"] - 1

    begin = Time(single_hp_test["t_begin"]).isot
    end = Time(single_hp_test["t_end"]).isot

    query = f"""
        SELECT 
          mean(displacement{hp_index}) as displacement, 
          mean(measuredForce{hp_index}) as measured_force
        FROM "lsst.sal.MTM1M3.hardpointActuatorData"
        WHERE time >= '{begin}Z'
        AND time <= '{end}Z'
        GROUP by time({sampling})
    """

    # Perform the query
    df = asyncio.run(client.influx_client.query(query))

    # Save information from the input dataframe into the queried dataframe
    # Broadcast *scalars* (no giant intermediate)
    meta = single_hp_test.to_dict()
    for k, v in meta.items():
        df[k] = v

    # Convert displacement from m to um
    df["displacement"] = df["displacement"] * METER_TO_MICROMETER

    return df


# Loop over all rows in test_windows_df
test_telemetry_parts = []
for _, row in test_windows_df.iterrows():
    df_part = query_tel_single_hp(client, row)
    test_telemetry_parts.append(df_part)

# Combine results
test_telemetry = pd.concat(test_telemetry_parts)
del test_telemetry_parts  # free list sooner to free memory

# Group up the tests and assign a new column to identify it
time_col = test_telemetry["t_begin"].sort_values()
time_diff = time_col.diff().gt(pd.Timedelta(seconds=30))
group_id = time_diff.cumsum()

test_telemetry["group"] = group_id.values

In [ ]:
def add_custom_time_format(ax: plt.Axes) -> None:
    """Convenient function to deal with time xaxis"""
    locator = mdates.MinuteLocator(interval=1)
    ax.xaxis.set_major_locator(locator)

    formatter = mdates.DateFormatter("%H:%M")
    ax.xaxis.set_major_formatter(formatter)


def add_plot_cosmetics(
    ax: plt.Axes, x_label: str, y_label: str, title: str, right_axis: bool = False
) -> None:
    """Convenient function to deal with cosmetics applied to every plot"""
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_ylim(TENSION_LOWER_LIMIT - 500, COMPRESSION_UPPER_LIMIT + 500)

    ax.set_title(title)
    ax.grid(":", alpha=0.2)

    if right_axis:
        ax.yaxis.set_label_position("right")
        ax.yaxis.tick_right()


def add_ranges_where_breakaway_should_happen(ax: plt.Axes) -> None:
    """
    Add two shaded areas representing where the breakaway mechanism
    must be triggered.
    """
    ax.axhspan(COMPRESSION_LOWER_LIMIT, COMPRESSION_UPPER_LIMIT, fc="gray", alpha=0.2)
    ax.axhspan(TENSION_LOWER_LIMIT, TENSION_UPPER_LIMIT, fc="gray", alpha=0.2)


def add_stiffness(ax: plt.Axes, df: pd.DataFrame) -> list[plt.Line2D]:
    """Add a dashed lines representing the calculated and the spec stiffness"""

    l1 = ax.plot(
        df.displacement,
        df.displacement * SPEC_STIFFNESS,
        ":",
        color="k",
        label=f"Spec Stiff. [{SPEC_STIFFNESS} N/um]",
    )

    p = get_stiffness_polynomial(df)
    if p is None:
        return [l1]

    idx = df.hardpoint.iloc[0] - 1
    l2 = ax.plot(
        df.displacement,
        np.polyval(p, df.displacement),
        c=f"C{idx}",
        ls="--",
        label=f"Meas. Stiff. [{np.round(p[0])} N/um]",
    )

    return [l1, l2]


def add_time_vertical_lines(ax: plt.Axes, df: pd.DataFrame) -> None:
    """Add vertical lines for t_begin, t_positive, t_negative, and t_end"""
    for col_name in ["t_begin", "t_positive", "t_negative", "t_end"]:
        timestamp = get_timestamp_from_dataframe(df, col_name)
        ax.axvline(timestamp, ls="--", lw=0.5, c="black")


def get_stiffness_polynomial(df: pd.DataFrame) -> Optional[np.ndarray]:
    """Fit a polynomial to the data near the minimum measured forces"""
    if df.measured_force.abs().min() > MEASURED_FORCE_MAXIMUM_TOLERANCE:
        reason = df.end_reason.iloc[0]
        hp = df.hardpoint.iloc[0]
        print(f"Warning - Not fitting stiffness for HP{hp}")
        return None

    df = df[df.displacement.abs() <= DISPLACEMENT_CROP_RANGE_FOR_FIT]
    poly = np.polyfit(df.displacement, df.measured_force, deg=1)

    return poly


def get_timestamp_from_dataframe(df: pd.DataFrame, column_name: str) -> pd.Timestamp:
    """
    Retrieve timestamps from a dataframe.
    The `column_name` is expected to be a column that contains timestamps.
    """
    return round_seconds(df[column_name].iloc[0])


def round_seconds(obj: pd.Timestamp) -> pd.Timestamp:
    """
    Round the timestamps to seconds.
    """
    if obj.microsecond >= 500000:
        obj += pd.Timedelta(seconds=1)
    return obj.replace(microsecond=0)


def select_region_where_forces_are_changing(
    df: pd.DataFrame, column_start: str, column_end: str
) -> pd.DataFrame:
    """Filter out the regions where the forces are changing in a linear fashion"""
    # Get the timestamps near the interesting region
    time_start = get_timestamp_from_dataframe(df, column_start)
    time_end = get_timestamp_from_dataframe(df, column_end)

    # Filter out our dataframe considering the time
    df = df[(df.index >= time_start) * (df.index <= time_end)].copy()

    # Find force intersection nearest to zero
    minimum_measured_force_index = df.measured_force.abs().idxmin()
    displacement_where_force_is_minimum = df.displacement.loc[
        minimum_measured_force_index
    ]

    if df.measured_force.abs().min() > MEASURED_FORCE_MAXIMUM_TOLERANCE:
        reason = df.end_reason.iloc[0]
        hp = df.hardpoint.iloc[0]
        print(
            f"Warning - Could not find valid test data "
            f"when {HardpointTest(reason).name} for HP{hp} "
            f"found minimum absolute measured forces {df.measured_force.abs().min()}."
        )

    # Put the zero of the displacement near the minimum measured force
    df.displacement = df.displacement - displacement_where_force_is_minimum

    # Filter out based on the displacement values
    df = df[df.displacement.abs() <= DISPLACEMENT_CROP_RANGE]

    return df


def plot_timeline(ax: plt.Axes, df: pd.DataFrame, idx: int) -> None:
    """
    Plot the whole test sequence for an individual hardpoint.
    """
    # Plot the timeline containing the
    label = (
        f"Breakaway Results\n"
        f" - Compression = {np.round(df.measured_force.max())} N\n"
        f" - Tension = {np.round(df.measured_force.min())} N"
    )

    ax.plot(df.index, df.measured_force, color=f"C{idx}", label=label)

    # Add details and cosmetics
    warning = (
        ""
        if df.end_reason.iloc[0] != HardpointTest.NOTTESTED
        else "- Warning - Ended with NOTTESTED"
    )
    t_begin = get_timestamp_from_dataframe(df, "t_begin")
    t_end = get_timestamp_from_dataframe(df, "t_end")
    title = (
        f"Breakaway Results for HardPoint {idx + 1} {warning}\n"
        f"From {t_begin:%Y-%m-%d %H:%M:%S} to {t_end:%Y-%m-%d %H:%M:%S}"
    )

    add_plot_cosmetics(ax, "Time [UTC]", "Measured Force [N]", title)
    add_ranges_where_breakaway_should_happen(ax)
    add_time_vertical_lines(ax, df)
    add_custom_time_format(ax)
    ax.legend(fontsize=9)


def plot_positive(ax: plt.Axes, df: pd.DataFrame, idx: int) -> None:
    """
    Plot the measured forces versus the displacement for when moving the
    hardpoint in the positive (compression) direction. The slant of the curve
    near zero corresponds to the stiffness of the hardpoint in that direction.
    """
    # Select relevant area
    df = select_region_where_forces_are_changing(df, "t_positive", "t_negative")

    # Plot the displacement
    ax.plot(df.displacement, df.measured_force, color=f"C{idx}", alpha=0.5)

    # Show the ranges where the breakaway is expected to happen
    add_ranges_where_breakaway_should_happen(ax)
    add_plot_cosmetics(ax, "Displacement [um]", "Measured Force [N]", "Moving Positive")
    add_stiffness(ax, df)
    ax.legend(fontsize=9)


def plot_negative(ax: plt.Axes, df: pd.DataFrame, idx: int) -> None:
    """
    Plot the measured forces versus the displacement for when moving the
    hardpoint in the positive (compression) direction. The slant of the curve
    near zero corresponds to the stiffness of the hardpoint in that direction.
    """
    # Select relevant area
    df = select_region_where_forces_are_changing(df, "t_negative", "t_end")

    # Plot the displacement
    ax.plot(df.displacement[::-1], df.measured_force[::-1], color=f"C{idx}", alpha=0.5)

    # Show the ranges where the breakaway is expected to happen
    add_plot_cosmetics(
        ax,
        "Displacement [um]",
        "Measured Force [N]",
        "Moving Negative",
        right_axis=True,
    )
    add_ranges_where_breakaway_should_happen(ax)
    add_stiffness(ax, df)
    ax.legend(fontsize=9)


def plot_hp_breakaway_results_for_group(df: pd.DataFrame, group_id: int) -> None:
    """Main function where all the magic happens when plotting the data"""

    # Select data to plot
    df = df[df.group == group_id].copy()

    # Get number of hardpoints that were tested
    number_of_hardpoints = df["hardpoint"].unique().size

    # Create a new figure with space for all the hardpoint plots
    fig = plt.figure(
        num=f"M1M3 HP Tests - Group {group_id} for {day_obs}",
        figsize=(9, 6 * number_of_hardpoints),  # (width, height)
    )

    # Create an outer grid to populate with plots
    gs_outer = gs.GridSpec(number_of_hardpoints, 1, figure=fig, hspace=0.4)

    # Loop over all the hardpoints and add plots
    for index in range(number_of_hardpoints):
        hp_df = df[df.hardpoint == (index + 1)]
        gs_inner = gs.GridSpecFromSubplotSpec(
            2, 2, subplot_spec=gs_outer[index], hspace=0.5, wspace=0.1
        )

        ax_timeline = fig.add_subplot(gs_inner[0, :])
        plot_timeline(ax_timeline, hp_df, index)

        ax_positive = fig.add_subplot(gs_inner[1, 0])
        plot_positive(ax_positive, hp_df, index)

        ax_negative = fig.add_subplot(gs_inner[1, 1])
        plot_negative(ax_negative, hp_df, index)

    plt.show()


# Create a dropdown menu to pick one of the test executions
dropdown = widgets.Dropdown(
    options=test_telemetry.group.unique(),
    value=0,
    description="Select HP test group:",
    style={"description_width": "auto"},
)

# Create a container for our plots
output = widgets.Output()


# Function that is triggered every time we select a new value in the dropdown menu
def update_plot(change):
    with output:
        output.clear_output(wait=True)
        plot_hp_breakaway_results_for_group(test_telemetry, change["new"])


# Link the dropdown menu with the function above
dropdown.observe(update_plot, names="value")

# Trigger initial plot
update_plot({"new": dropdown.value})

# Display the components
widgets.VBox([dropdown, output])

## Debug part

In [ ]:
def replace_enum(x):
    return HardpointTest(x).name

test_status_df.apply(np.vectorize(replace_enum))